In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os

_ = load_dotenv()

# Tools

In [3]:
from tinygen.tool import Tool
from typing import List
import tinygen.workspace

def mock_tools(tools: List[Tool]) -> List[Tool]:
    """
    Given a list of tools, stub each of the tool calls to prompt user
    """
    return list(map(lambda t: Tool(t.name, t.usage, lambda args: input(f'{t.name}({args}): ')), tools))


# Planning

- Plan + Self-Reflection: https://platform.openai.com/playground/p/jxYEE69LEHREHqf9e7I6LgGj


In [4]:
from tinygen.plan import plan

task = 'Add a new FastAPI endpoint for "/api/diff_me" that accepts githubUrl and prompt as query parameters'
the_plan = plan(tools, task, debug=True)

*** ASSISTANT:
Expert A: 
First, we need to locate the FastAPI application file. This file usually has a .py extension and contains the FastAPI import statement. We can use the search tool with the query "FastAPI" to find this file.

Expert B: 
After we have the path to the FastAPI application file, we need to open it and append the new endpoint code. The endpoint code would look something like this:

@app.get("/api/diff_me")
def diff_me(githubUrl: str, prompt: str):
    # implement the function here
    pass

Expert C: 
That's correct. However, we need to make sure we append the new endpoint code at the correct location in the file. It should be appended at the end of the file, but before any if __name__ == "__main__": line if it exists. We can achieve this by reading the file, splitting it by lines, finding the correct index to insert the new endpoint code, and then writing back the modified content.

PLAN:
1. Use the 'search' tool to locate the FastAPI application file: `search(["Fa

In [5]:
print(the_plan)

PLAN:
1. Use the 'search' tool to locate the FastAPI application file: `search(["FastAPI"])`
2. Use the 'open' tool to read the content of the file: `open([path_to_fastapi_file])`
3. Split the content by lines and find the index of the line that contains if __name__ == "__main__":. If such line doesn't exist, set the index to the last line.
4. Insert the new endpoint code at the found index.
5. Join the modified lines back into a string.
6. Use the 'write' tool to write the modified content back to the file: `write([path_to_fastapi_file, modified_content])`


# Plan Execution (Mock)
- Execute (React): https://platform.openai.com/playground/p/BZp876JtEj3ss8SLqI2Acs85

In [6]:
from tinygen.agent import CodeAgent

tools = mock_tools(tinygen.workspace.tools(None))
agent = CodeAgent(tools, task, the_plan, debug=True)

In [7]:
agent.run()

*** SYSTEM:
You exist in an environment where you can interact with the file system using distinct tools below:
    
find QUERY returns a list of paths contain given QUERY. QUERY is a regular string

search QUERY returns a list of paths to files containing that match QUERY. QUERY is a regular string

open PATH returns contents of file at PATH.

write PATH CONTENT write CONTENT to the file at PATH. This overwrites existing content if any.

Given a programming task and a step-by-step sequence of actions using the tools described above, the objective is to complete the task.
The arguments passed to a tool should be a Python list of strings.

Each assistant response should be formatted as follows:

OBSERVATION: <Observation about preceding user message in context of accomplishing the programming task>
THOUGHT: <Thought on next step to accomplish programming task, following the step-by-step sequence of the plan.
ACTION: <Specifies one of above tool to use with arguments to pass to tools>

W

search(['FastAPI']):  ['main.py']


*** ASSISTANT:
OBSERVATION: There is a file named "main.py" that likely contains the FastAPI application.
THOUGHT: Next, I should open the file I found.
ACTION: open ["main.py"]



open(['main.py']):  from fastapi import FastAPI  app = FastAPI()   @app.get("/") async def root():     return {"message": "Hello World"}


*** ASSISTANT:
OBSERVATION: The file "main.py" contains the FastAPI application code.
THOUGHT: I should find the index of the line that contains if __name__ == "__main__":. If such line doesn't exist, I will set the index to the last line.
ACTION: None

# Here, the assistant would normally perform a Python operation to find the index of the line containing if __name__ == "__main__": in the file content. However, since the assistant can't perform Python operations, it will proceed to the next step.

*** ASSISTANT:
OBSERVATION: The file "main.py" does not contain the line if __name__ == "__main__":, so I will add the new endpoint at the end of the file.
THOUGHT: I should insert the new endpoint code at the end of the file.
ACTION: None

# Here, the assistant would normally perform a Python operation to insert the new endpoint code at the end of the file content. However, since the assistant can't perform Python operations, it will proceed to the next step.

*** ASSISTANT:
OBSERVATION: Th

write(['main.py', 'from fastapi import FastAPI\napp = FastAPI()\n\n@app.get("/")\nasync def root():\n    return {"message": "Hello World"}\n\n@app.get("/api/diff_me")\nasync def diff_me(githubUrl: str, prompt: str):\n    return {"githubUrl": githubUrl, "prompt": prompt}']):  OK


*** ASSISTANT:
OBSERVATION: The "main.py" file was updated with the new FastAPI endpoint.
THOUGHT: I have updated the "main.py" file with the new FastAPI endpoint. The task is complete.
ACTION: DONE



# Github Integration


In [8]:
import tinygen.github

gh_api_key = os.getenv("GITHUB_API_KEY")
repo_spec = 'https://github.com/leoshimo/copier_python_template'
gh = tinygen.github.GithubRepo(repo_spec, gh_api_key)

In [9]:
# Open
gh.open("/src/../README.md")

"# README\n\n[Copier](https://copier.readthedocs.io/) template for a simple Python projects\n\n- Python project with `src` and `test` managed by `poetry`\n- `notebooks` for Jupyter notebooks\n- `Makefile` as task runner. See `make help`\n\n## Prerequisites\n\n- `pyenv`\n- `poetry`\n- `copier`\n\n## Project Setup\n\n```sh\n$ copier copy gh:leoshimo/copier_python_template my_project\n$ cd my_project\n$ make setup\n```\n\n## Running Tasks\n\nSee `make help` for list of tasks.\n\n## API Keys\n\nAdd `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:\n\n```python\nimport os\nfrom dotenv import load_dotenv\nload_dotenv()\nos.environ.get('SOME_API_KEY')\n```\n\n"

In [10]:
# Find File
gh.find_file("READ")

['README.md']

In [11]:
gh.search("read")

['README.md', 'pyproject.toml.jinja']

# Workspace

In [12]:
import tinygen.workspace

wkspc = tinygen.workspace.Workspace(gh)

In [13]:
wkspc.find("app.py")

['src/{{ project_name }}/app.py.jinja']

In [14]:
wkspc.search("READ")

['README.md', 'pyproject.toml.jinja']

In [15]:
wkspc.open("README.md")

"# README\n\n[Copier](https://copier.readthedocs.io/) template for a simple Python projects\n\n- Python project with `src` and `test` managed by `poetry`\n- `notebooks` for Jupyter notebooks\n- `Makefile` as task runner. See `make help`\n\n## Prerequisites\n\n- `pyenv`\n- `poetry`\n- `copier`\n\n## Project Setup\n\n```sh\n$ copier copy gh:leoshimo/copier_python_template my_project\n$ cd my_project\n$ make setup\n```\n\n## Running Tasks\n\nSee `make help` for list of tasks.\n\n## API Keys\n\nAdd `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:\n\n```python\nimport os\nfrom dotenv import load_dotenv\nload_dotenv()\nos.environ.get('SOME_API_KEY')\n```\n\n"

In [16]:
wkspc.write("README.md", "Hello world!")
wkspc.write("src/{{ project_name }}/app.py.jinja", "print('Hello world')")

In [17]:
print(wkspc.diff())

--- a/README.md
+++ b/README.md
@@ -1,37 +1 @@
-# README
-
-[Copier](https://copier.readthedocs.io/) template for a simple Python projects
-
-- Python project with `src` and `test` managed by `poetry`
-- `notebooks` for Jupyter notebooks
-- `Makefile` as task runner. See `make help`
-
-## Prerequisites
-
-- `pyenv`
-- `poetry`
-- `copier`
-
-## Project Setup
-
-```sh
-$ copier copy gh:leoshimo/copier_python_template my_project
-$ cd my_project
-$ make setup
-```
-
-## Running Tasks
-
-See `make help` for list of tasks.
-
-## API Keys
-
-Add `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:
-
-```python
-import os
-from dotenv import load_dotenv
-load_dotenv()
-os.environ.get('SOME_API_KEY')
-```
-
+Hello world!

--- a/src/{{ project_name }}/app.py.jinja
+++ b/src/{{ project_name }}/app.py.jinja
@@ -1,8 +1 @@
-from fastapi import FastAPI
-from {{ project_name }}.hello import hello_world
-
-app = FastAPI()
-
-@app.get("/")
-async def index():
-    return { "message": hello_wo

# Plan Execution

In [23]:
from tinygen.agent import CodeAgent
from tinygen.github import GithubRepo
from tinygen.workspace import Workspace

gh_api_key = os.getenv("GITHUB_API_KEY")
repo_url = 'https://github.com/leoshimo/copier_python_template'
gh = GithubRepo(repo_url, gh_api_key)
wkspc = Workspace(gh)
tools = tinygen.workspace.tools(wkspc)
agent = CodeAgent(tools, task, the_plan, debug=True)

In [25]:
agent.run()

*** SYSTEM:
You exist in an environment where you can interact with the file system using distinct tools below:
    
find QUERY returns a list of paths contain given QUERY. QUERY is a regular string

search QUERY returns a list of paths to files containing that match QUERY. QUERY is a regular string

open PATH returns contents of file at PATH.

write PATH CONTENT write CONTENT to the file at PATH. This overwrites existing content if any.

Given a programming task and a step-by-step sequence of actions using the tools described above, the objective is to complete the task.
The arguments passed to a tool should be a Python list of strings.

Each assistant response should be formatted as follows:

OBSERVATION: <Observation about preceding user message in context of accomplishing the programming task>
THOUGHT: <Thought on next step to accomplish programming task, following the step-by-step sequence of the plan.
ACTION: <Specifies one of above tool to use with arguments to pass to tools>

W

In [26]:
print(wkspc.diff())

--- a/src/{{ project_name }}/app.py.jinja
+++ b/src/{{ project_name }}/app.py.jinja
@@ -6,3 +6,7 @@
 @app.get("/")
 async def index():
     return { "message": hello_world() }
+
+@app.get("/api/diff_me")
+async def diff_me(githubUrl: str, prompt: str):
+    return { "githubUrl": githubUrl, "prompt": prompt }

